This assignment is inspired by: 

- https://www.kaggle.com/code/carlmcbrideellis/an-introduction-to-xgboost-regression
- https://www.kaggle.com/code/dansbecker/xgboost/notebook

In this assignment we will apply XGBoost Regression techniques to predict house prices, based on the famous Kaggle Dataset https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Step 1 is to download the dataset.

In [6]:
#=========================================================================
# load up the libraries
#=========================================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import scipy.stats as stats
import numpy as np

# Change settings for viewing records 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#=========================================================================
# read in the data
#=========================================================================
train_data = pd.read_csv('train.csv',index_col=0)
test_data  = pd.read_csv('test.csv',index_col=0)

# Feature Exploration, Selection and Model Building

In [66]:
## Explore the data 

# print(train_data.head())
# print(train_data.info())
# print(train_data.describe())

# Divide into train and validate (80 train and 20 validate)
train_df, validate_df = train_test_split(train_data, test_size=0.2, random_state=42)


# Drop columns with missing values > 10%
missing_threshold = 0.1 * len(train_df)
train_df = train_df.dropna(thresh=missing_threshold, axis=1)
validate_df = validate_df[train_df.columns]


# Retain columns that are statistically significant (T-test for categorical columns and correlation for numerical columns)
# T-test for categorical columns
categorical_cols = train_df.select_dtypes(include='object').columns
significant_categorical_cols = []
for col in categorical_cols:
    t_stat, p_value = stats.ttest_ind(train_df[train_df['SalePrice'] == 0][col], train_df[train_df['SalePrice'] == 1][col])
    if p_value < 0.05:
        significant_categorical_cols.append(col)



# Correlation for numerical columns
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns
corr_matrix = train_df[numerical_cols].corr()
significant_numerical_cols = [col for col in numerical_cols if abs(corr_matrix['SalePrice'][col]) > 0.1]

significant_cols = significant_categorical_cols + significant_numerical_cols


train_df = train_df[significant_cols]
validate_df = validate_df[significant_cols]

# # Separate features and target variable
X_train = train_df.drop('SalePrice', axis=1)
y_train = train_df[['SalePrice']]
X_validate = validate_df.drop('SalePrice', axis=1)
y_validate = validate_df[['SalePrice']]

# Train XGBoost model
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)

# Test the model on the validate set
y_pred = xgb_model.predict(X_validate)

# Evaluate model performance
mse = mean_squared_error(y_validate, y_pred)
print(f'Mean Squared Error on Validate Set: {mse}')

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

grid_search = GridSearchCV(XGBRegressor(), param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_

# Retrain the model on the entire train dataset (train + validate) with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params)

X_combined = pd.concat([X_train, X_validate], ignore_index=True)
y_combined = pd.concat([y_train, y_validate], ignore_index=True)
best_xgb_model.fit(X_combined, y_combined)


Mean Squared Error on Validate Set: 1002922309.9236392


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# Predict on Test Data Set

In [91]:
test_data = test_data[test_data.columns[test_data.columns.isin(X_combined.columns)]]
test_data.info()
test_data['BsmtFinSF1'].astype('float64')
test_data['BsmtUnfSF'].astype('float64')
test_data['TotalBsmtSF'].astype('float64')
test_data['BsmtFullBath'].astype('float64')
y_test = best_xgb_model.predict(test_data)
y_test = pd.DataFrame(y_test)
y_test.columns = ['Predicted_Sales_Price']
y_test.to_csv('submission.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1459 entries, 1461 to 2919
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontage    1459 non-null   float64
 1   LotArea        1459 non-null   int64  
 2   OverallQual    1459 non-null   int64  
 3   YearBuilt      1459 non-null   int64  
 4   YearRemodAdd   1459 non-null   int64  
 5   MasVnrArea     1459 non-null   float64
 6   BsmtFinSF1     1459 non-null   int64  
 7   BsmtUnfSF      1459 non-null   int64  
 8   TotalBsmtSF    1459 non-null   int64  
 9   1stFlrSF       1459 non-null   int64  
 10  2ndFlrSF       1459 non-null   int64  
 11  GrLivArea      1459 non-null   int64  
 12  BsmtFullBath   1459 non-null   int64  
 13  FullBath       1459 non-null   int64  
 14  HalfBath       1459 non-null   int64  
 15  BedroomAbvGr   1459 non-null   int64  
 16  KitchenAbvGr   1459 non-null   int64  
 17  TotRmsAbvGrd   1459 non-null   int64  
 18  Fireplaces

Finally, can you run it on your test set?

array([128218.39 , 151068.9  , 175290.16 , 185664.92 , 194694.75 ,
       177672.94 , 168205.89 , 163902.75 , 181029.33 , 127697.125],
      dtype=float32)

In [35]:
best_xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [36]:

# Retrain the model on the entire train dataset (train + validate)




XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [40]:

print(y_train == y_validate)
#y_combined.head(100)
# data_types_train = X_train.dtypes

# # Apply the data types from the training set to the test set
# test_data = test_data.astype(data_types_train)
# y_test = xgb_model.predict(test_data)

      SalePrice  SalePrice
Id                        
893        True       True
1106       True       True
414        True       True
523        True       True
1037       True       True
615        True       True
219        True       True
1161       True       True
650        True       True
888        True       True
577        True       True
1253       True       True
1062       True       True
568        True       True
1109       True       True
1114       True       True
169        True       True
1103       True       True
1121       True       True
68         True       True
1041       True       True
454        True       True
671        True       True
1095       True       True
193        True       True
124        True       True
416        True       True
278        True       True
434        True       True
1318       True       True
185        True       True
555        True       True
1174       True       True
77         True       True
907        True       True
6

In [ ]:
Can you score your solution offline and see how it does?

In [ ]:
# read in the ground truth file
solution   = pd.read_csv(<your solution file>)
y_true     = solution["SalePrice"]

from sklearn.metrics import mean_squared_log_error
RMSLE = np.sqrt( mean_squared_log_error(y_true, predictions) )
print("The score is %.5f" % RMSLE )

Finally, use the below block to prepare your submission

In [ ]:
output = pd.DataFrame({"Id":test_data.index, "SalePrice":predictions})
output.to_csv('submission.csv', index=False)

### <center style="background-color:Gainsboro; width:60%;">Feature importance</center>
Let us also take a very quick look at the feature importance too:

In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 16})

fig, ax = plt.subplots(figsize=(12,6))
plot_importance(regressor, max_num_features=8, ax=ax)
plt.show();

Where here the `F score` is a measure "*...based on the number of times a variable is selected for splitting, weighted by the squared improvement to the model as a result of each split, and averaged over all trees*." [1] 

Note that these importances are susceptible to small changes in the training data, and it is much better to make use of ["GPU accelerated SHAP values"](https://www.kaggle.com/carlmcbrideellis/gpu-accelerated-shap-values-jane-street-example), incorporated with version 1.3 of XGBoost.

Can you follow the above guide use SHAP values instead of F Score?

In [1]:
# code here

### <center style="background-color:Gainsboro; width:60%;">Appendix: The RMSLE evaluation metric</center>
From the competition [evaluation page](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation) we see that the metric we are using is the root mean squared logarithmic error (RMSLE), which is given by

$$ {\mathrm {RMSLE}}\,(y, \hat y) = \sqrt{ \frac{1}{n} \sum_{i=1}^n \left(\log (1 + \hat{y}_i) - \log (1 + y_i)\right)^2} $$

where $\hat{y}_i$ is the predicted value of the target for instance $i$, and $y_i$
is the actual value of the target for instance $i$.

It is important to note that, unlike the RMSE, the RMSLE is asymmetric; penalizing much more the underestimated predictions than the overestimated predictions. For example, say the correct value is $y_i = 1000$, then underestimating by 600 is almost twice as bad as overestimating by 600:

In [ ]:
def RSLE(y_hat,y):
    return np.sqrt((np.log1p(y_hat) - np.log1p(y))**2)

print("The RMSLE score is %.3f" % RSLE( 400,1000) )
print("The RMSLE score is %.3f" % RSLE(1600,1000) )

The asymmetry arises because 

$$ \log (1 + \hat{y}_i) - \log (1 + y_i) =  \log \left( \frac{1 + \hat{y}_i}{1 + y_i} \right) $$

so we are essentially looking at ratios, rather than differences such as is the case of the RMSE. We can see the form that this asymmetry takes in the following plot, again using 1000 as our ground truth value:

In [ ]:
plt.rcParams["figure.figsize"] = (7, 4)
x = np.linspace(5,4000,100)
plt.plot(x, RSLE(x,1000))
plt.xlabel('prediction')
plt.ylabel('RMSLE')
plt.show()